In [42]:
import requests, csv
from bs4 import BeautifulSoup
import re
import pandas as pd
import tqdm
import numpy as np
from collections import OrderedDict

In [43]:
def movie_imdb(imdb_id):
    
    #SELECT SYNOPSIS, GENRES, RATING, NUMBER RATE, YEAR, TITLE, ORIGINAL TITLE, COUNTRY,
    #SELECT LANGUAGE, PRODUCTION, BUDGET, INCOME
    
    r1 = requests.get("https://www.imdb.com/title/"+imdb_id)
    bs = BeautifulSoup(r1.text)
    
    # Dictionnary for title, orginal_title, rating, num_rate
    
    info_dict = bs.find('script', {'type': 'application/ld+json'})
    info_dict = info_dict.text
    info_dict = OrderedDict(eval(info_dict))
    
    original_title = info_dict["name"]
    try :
        title = info_dict["alternateName"]
    except KeyError:
        title = info_dict["name"]
           
    rating = info_dict['aggregateRating']['ratingValue']
    num_rate = info_dict['aggregateRating']['ratingCount']
    
    
    #Dictionnary for budget, bud_currency, income, inc_currency
    
    info_dict_2 = bs.findAll('script', {'id': re.compile('__NEXT_DATA__')})
    info_dict_2 = [text.text for text in info_dict_2][0]
    info_dict_2 = OrderedDict(eval(info_dict_2, {'null': None, 'false':None, 'true':None}))
    info_dict_2_1 = info_dict_2['props']['urqlState']
    
    for key in info_dict_2_1.keys():
        try:
            budget = info_dict_2_1[key]['data']['title']['productionBudget']
            def_key = key
        except KeyError:
            continue
    
    if budget == None:
        budget = ""
        bud_currency = ""
    else : 
        budget = info_dict_2_1[def_key]['data']['title']['productionBudget']["budget"]["amount"]
        bud_currency = info_dict_2_1[def_key]['data']['title']['productionBudget']["budget"]["currency"]
        

    income = info_dict_2_1[def_key]['data']['title']['worldwideGross']

    if income == None:
        income = ""
        inc_currency = ""
    else : 
        income = info_dict_2_1[def_key]['data']['title']['worldwideGross']["total"]["amount"]
        inc_currency = info_dict_2_1[def_key]['data']['title']['worldwideGross']["total"]["currency"]
    
    
    year = bs.findAll('a', {'class': re.compile('ipc-link')})
    year = int([text.text for text in year][0])
    
    genre_bs = bs.findAll('a', {'class': re.compile('ipc-metadata-list-item')})
    genre_bs = [text.text for text in genre_bs]
    movie_genre = []
    genres = ""
    genre_list = ["Action", "Adult", "Adventure", "Animation", "Biography", "Comedy", "Crime", "Documentary", 
              "Drama", "Family", "Fantasy", "Film-Noir", "History", "Horror", "Music", "Musical", "Mystery", 
              "News", "Reality-TV", "Romance", "Sci-Fi", "Short", "Sport", "Thriller", "War", "Western"]
    for genre in genre_bs:
        if genre in genre_list:
            movie_genre.append(genre)
        genres = " | ".join(movie_genre)
    
    syn = bs.findAll('div', {'class': 'ipc-html-content ipc-html-content--base'})[0]
    syn = [text.text for text in syn]
    synopsis = syn[0].split(" —")[0]
    
    countries = bs.findAll('a', {'href': re.compile('country_of_origin')})
    countries = [text.text for text in countries]
    country = []
    for state in countries:
        if state == "United States":
            country.append("USA")
        elif state == "United Kingdom":
            country.append("UK")
        else :
            country.append(state)
    country = " | ".join(country)
    
    language = bs.findAll('a', {'href': re.compile('primary_language')})
    language = [text.text for text in language]
    language = " | ".join(language)
    
    
    
    #SELECT CASTING, DIRECTOR, WRITERS
    
    r2 = requests.get("https://www.imdb.com/title/"+imdb_id+"/fullcredits")
    bs2 = BeautifulSoup(r2.text)

    #select casting
    cast_list = bs2.find_all('table', {'class': 'cast_list'})
    cast = []
    for i in range(1, 30):
        try:
            cast.append((str(cast_list).split('img alt="')[i]).split('" class="')[0])
            casting = str(cast)
            casting = re.sub("'", "", casting[1:-1])
        except IndexError:
            break
    
    #select director
    try:
        direct = bs2.find('table', {'class': 'simpleTable simpleCreditsTable'}) \
                .find_all('td', {'class': 'name'})
        director = [text.text for text in direct]
        director = str(director)[1:-1].replace("\\n ", "").replace("'", "")
    except (IndexError, AttributeError):
        director = ""

    
    #select writers
    try:
        write = bs2.find_all('table', {'class': 'simpleTable simpleCreditsTable'})[1] \
                .find_all('td', {'class': 'name'})
        writers = [text.text for text in write]
        writers = str(writers)[1:-1].replace("\\n ", "").replace("'", "")
    except (IndexError, AttributeError):
        writers = ""
    
    
    
    #SELECT RUNTIME
    
    r3 = requests.get("https://www.imdb.com/title/"+imdb_id+"/technical")
    bs3 = BeautifulSoup(r3.text)

    time = bs3.find('table', {'class': 'dataTable labelValueTable'}).find_all('tr', {'class': 'odd'})
    runtime = [text.text for text in time]
    try:
        if ("min (" in (re.sub('\n', '', runtime[0]))) and (" min)" in (re.sub('\n', '', runtime[0]))):
            duration = int((re.sub('\n', '', runtime[0]).split("min (")[1]).split(" min)")[0].strip())
        elif "hr (" in (re.sub('\n', '', runtime[0])): 
            duration = int((re.sub('\n', '', runtime[0]).split("hr (")[1]).split(" min)")[0].strip())
        else :
            duration = int((re.sub('\n', '', runtime[0]).split("Runtime")[1]).split(" min ")[0].strip())
    except ValueError:
        duration = ""
        
        
    
    #SELECT PRODUCTION
    
    r4 = requests.get("https://www.imdb.com/title/"+imdb_id+"/companycredits")
    bs4 = BeautifulSoup(r4.content)
    prod = bs4.find('ul', {'class': 'simpleList'})
    prod = prod.text
    if "(" in prod:
        prod = re.sub(r'\([^)]*\)', '', prod)
    prod = prod.replace("\n ", "")
    prod = prod.strip().split("\n\n")
    productions = []
    for i in prod:
        productions.append(i.strip())
    production = " | ".join(productions)
    
    
    return imdb_id, "", "", title, original_title, year, director, casting, genres, duration, country, language, \
    writers, production, synopsis, rating, num_rate, budget, bud_currency, income, inc_currency

In [44]:
df = pd.read_excel('to_scrap.xlsx')

In [45]:
imdb_id = df["imdb"].values

In [113]:
imdb_id

array(['tt9844322', 'tt10954652'], dtype=object)

In [46]:
movies = []

In [47]:
for movie in tqdm.tqdm(imdb_id):
    movies.append(movie_imdb(movie))

100%|██████████| 17/17 [01:02<00:00,  3.67s/it]


In [48]:
movies[1]

('tt10954652',
 '',
 '',
 'Old',
 'Old',
 2021,
 'M. Night Shyamalan',
 'Gael García Bernal, Vicky Krieps, Rufus Sewell, Alex Wolff, Thomasin McKenzie, Abbey Lee, Nikki Amuka-Bird, Ken Leung, Eliza Scanlen, Aaron Pierre, Embeth Davidtz, Emun Elliott, Alexa Swinton, Gustaf Hammarsten, Kathleen Chalfant, Francesca Eastwood, Nolan River, Luca Faustino Rodriguez, Mikaya Fisher, Kailen Jude, M. Night Shyamalan, Matthew Shear, Daniel Ison, Jeffrey Holsman, Margaux Da Silva, John Twohy',
 'Drama | Mystery | Thriller',
 108,
 'USA',
 'English',
 'M. Night Shyamalan, Pierre-Oscar Lévy, Frederik Peeters',
 'Universal Pictures | Blinding Edge Pictures | Perfect World Pictures',
 'This summer, visionary filmmaker M. Night Shyamalan unveils a chilling, mysterious new thriller about a family on a tropical holiday who discover that the secluded beach where they are relaxing for a few hours is somehow causing them to age rapidly - reducing their entire lives into a single day.',
 5.9,
 23645,
 '',
 ''

In [116]:
columns_name = ["imdb_id", "vu", "à voir", "title", "original_title", "year", "director", "casting", "genres", "duration", "country", "language", \
"writers", "production", "synopsis", "rating", "num_rate", "budget", "bud_currency", "income", "inc_currency"]

In [117]:
result = pd.DataFrame(list(movies), columns = columns_name)
result = result.set_index("imdb_id")

In [118]:
result.to_csv("movies_scraped.csv")

In [23]:
def genre_imdb(imdb_id):
    
    r1 = requests.get("https://www.imdb.com/title/"+imdb_id)
    bs = BeautifulSoup(r1.text)
    
    genre_bs = bs.findAll('a', {'class': re.compile('ipc-metadata-list-item')})
    genre_bs = [text.text for text in genre_bs]
    movie_genre = []
    genres = ""
    genre_list = ["Action", "Adult", "Adventure", "Animation", "Biography", "Comedy", "Crime", "Documentary", 
              "Drama", "Family", "Fantasy", "Film-Noir", "History", "Horror", "Music", "Musical", "Mystery", 
              "News", "Reality-TV", "Romance", "Sci-Fi", "Short", "Sport", "Thriller", "War", "Western"]
    for genre in genre_bs:
        if genre in genre_list:
            movie_genre.append(genre)
        genres = " | ".join(movie_genre)
    
    return imdb_id, genres

In [2]:
df = pd.read_excel('id_title.xlsx')

In [4]:
imdb_id = df["imdb_id"].values

In [37]:
movies = []

In [12]:
imdb_id[48604]

'tt0041555'

In [38]:
for movie in tqdm.tqdm(imdb_id[82584:]):
    movies.append(genre_imdb(movie))

100%|██████████| 3398/3398 [1:29:33<00:00,  1.58s/it]


In [39]:
columns_name = ["imdb_id", "genres"]

In [40]:
result = pd.DataFrame(list(movies), columns = columns_name)
result = result.set_index("imdb_id")

In [41]:
result.to_excel("new_genre2.xlsx")

### Other tests of Code

In [ ]:
    title = bs.findAll('h1', {'class': re.compile('^TitleHeader')})
    title = [text.text for text in title][0]
    
    if len(bs.findAll('div', {'class': re.compile('^OriginalTitle')})) == 0:
        original_title = title
    else:
        original_title = bs2[1].findAll('div', {'class': re.compile('^OriginalTitle')})
        original_title = [text.text for text in or_title][0].split(": ")[1]
        
    directors = []
    for director in info_dict["director"]:
        directors.append(director["name"])
    director = ", ".join(directors)